Looking at **ridge regression** analysis on the Qgiv conversion data to see if we can improve accuracy.

In [6]:
%pylab inline
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
matplotlib.style.use('ggplot')

Populating the interactive namespace from numpy and matplotlib


In [5]:
analytic_base = pd.read_csv('~/Repositories/datasets/analytic_base.csv')
analytic_qgiv = pd.read_csv('~/Repositories/datasets/analytic_qgiv_stats.csv')

ab = analytic_base[['id', 'org', 'form', 'timestamp', 'visits', 'mobile_visits', 'don_form_trans_count', 'don_form_trans_vol']]
aq = analytic_qgiv[['id', 'base', 'org', 'total_visits', 'opt_fields', 'req_fields', 'donation_active', 'amounts_system', 'multirestriction_system', 'restrictions', 'pledges_count', 'pledge_active', 'permit_anonymous', 'permit_mobile', 'permit_other_amount', 'enable_donorlogins', 'collect_captcha']]
d = pd.merge(ab, aq, left_on="id", right_on="base")

In [13]:
# filter out not visited observations
data = d[d.visits > 0]
# add conversion
conversion = pd.DataFrame({'conversion':data["don_form_trans_count"]/data["total_visits"]*100})
# merge conversion w/ the rest of the data
data = pd.concat([data, conversion], axis=1)
# add day_of_month & month
date_data = pd.DataFrame({
        'day': pd.to_datetime(data.timestamp).apply(lambda x: x.day), 
        'month': pd.to_datetime(data.timestamp).apply(lambda x: x.month)})
# merge date data w/ the rest of the data
data = pd.concat([data, date_data], axis=1)

# Ridge regression analysis

Start with a straight forward fit w/out bases functions

In [16]:
features = ['opt_fields', 'multirestriction_system', 'restrictions', 'pledges_count', 'pledge_active', 'day', 'month']

In [17]:
# range of alpha values to try
alpha_ridge = [1e-15, 1e-10, 1e-8, 1e-4, 1e-3,1e-2, 1, 5, 10, 20]
fits = []

for a in alpha_ridge:
    ridgereg = Ridge(alpha=a, normalize=True)
    ridgereg.fit(data[features], data['conversion'])
    fits.append(ridgereg)

# Lasso analysis